# CmmD: Continual Multiplex network Module Detector

Python implementation of the algorithm originally proposed in ["The multilayer community structure of medulloblastoma" by Iker Núñez-Carpintero et. al.](https://www.sciencedirect.com/science/article/pii/S2589004221003333).

Implementation relies on [MolTi-DREAM](https://github.com/gilles-didier/MolTi-DREAM) for communities detection problem.

* Make sure to add molti-console as an executable program from any directory by adding it to your ~/.bashrc file. (This file may change depending on your OS ditribution)

## 0. Import libraries and define paths

In [ ]:
import os
import sys
import subprocess
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from scipy.spatial.distance import pdist, squareform

In [ ]:
def CmmD(nodelist=None, input_layers=None, resolution_start=None, resolution_end=None, interval=None, 
         distmethod="hamming", threads=1, destfile_community_analysis=None):
    
    """
    Compute CmmD multilayer community trajectory analysis for a set of given networks.

    Parameters
    ----------
    nodelist : list, optional
        A list with the unique nodes that we want to appear in the final output. If not given, all nodes of the multiplex will be in the final output (nodelist= NULL)
    input_layers : list
        A vector of strings containing the paths where the different network layers are located in the system. Networks should be a two column file representing the edges of the graph.
    resolution_start : float
        The first gamma resolution parameter to use in the different MolTi's analysis
    resolution_end : float
        The last gamma resolution parameter to use in the different MolTi's analysis.
    interval : float
        The interval of the resolution parameter to use. 
    distmethod : str, optional
        A distance method metric to use to compute the trajectories. Defaults to "hamming" for hamming distance, but accepts any other metric supplied by scipy.spatial.distance.pdist.
    threads : int, optional
        The number of threads to use for the computation of the distance matrix. Defaults to 1.
    destfile_community_analysis : str, optional
        The path to save Molti's output files. Defaults to "Output/".

    Returns
    -------
    A dictionary containing the following keys:
        gene_community_matrix: A matrix where the rows correspond to the different genes, and the columns to the different community structures. The values of the matrix are the cluster to which the gene belongs in the corresponding community structure.
        l_constant: A dictionary where the keys are the different community structures, and the values are the list of genes that belong to that community structure.
        distance_matrix: A matrix with the hamming distances between all pairs of genes.
    """
    if input_layers is None or len(input_layers) < 1:
        raise ValueError("ERROR: Input_layers argument must be a list of at least 1 network file")
    
    if not isinstance(resolution_end, (int, float)):
        raise ValueError("ERROR: Resolution parameter must be a number")
    
    if not isinstance(resolution_start, (int, float)):
        raise ValueError("ERROR: Resolution parameter must be a number")
    
    if not isinstance(interval, (int, float)):
        raise ValueError("ERROR: Interval value must be a number")
    
    if not isinstance(destfile_community_analysis, str):
        raise ValueError("ERROR: destfile_community_analysis expects a character string")
    
    if not isinstance(distmethod, str):
        raise ValueError("ERROR: distmethod expects a character string")
    
    if not isinstance(threads, int):
        raise ValueError("ERROR: Threads must be a number corresponding to the number of cores available to use")

    # Prepare inputs to generate the console order for MolTi's run.
    layers = " ".join(input_layers)
    print(f"Resolution parameter starts at: {resolution_start}")
    print(f"Resolution parameter ends at: {resolution_end}")
    
    resolution_interval = np.arange(resolution_start, resolution_end + interval, interval)
    desfile_vector = [f"{destfile_community_analysis}{res}.csv" for res in resolution_interval]
    
    print("Starting community analysis.")
    
    for i, current_resolution in enumerate(resolution_interval):
        current_destfile = desfile_vector[i]
        print(f"Resolution parameter: {current_resolution}")
        print(f"{current_resolution}: {os.popen('date').read().strip()}")
        system_order = f"molti-console -o {current_destfile} -p {current_resolution} {layers}"
        subprocess.run(system_order, shell=True)
    
    print("Reading MolTi output files. Calculating Gene/Community matrix")
    output_files = [f for f in os.listdir(destfile_community_analysis) if "_" not in f]
    
    alllists = []
    
    for output_file in output_files:
        with open(os.path.join(destfile_community_analysis, output_file), 'r') as file:
            red = file.readlines()
        
        cluster_ids = [i for i, line in enumerate(red) if "Cluster" in line]
        lista = []
        
        for j, st in enumerate(cluster_ids):
            if j == len(cluster_ids) - 1:
                en = len(red)
            else:
                en = cluster_ids[j + 1]
            current_cluster = red[st:en]
            current_cluster2 = current_cluster[:-2] if j != len(cluster_ids) - 1 else current_cluster[:-1]
            lista.append(current_cluster2[1:])
        
        alllists.append(lista)
    
    allgenes = list(set([gene for sublist in alllists for cluster in sublist for gene in cluster]))

    if nodelist:
        allgenes = list(set(allgenes).intersection(nodelist))
    
    print("Files read. Calculating Gene/Community matrix")
    res_matrix = np.zeros((len(allgenes), len(alllists) + 1), dtype=int)
    gene_indices = {gene: idx for idx, gene in enumerate(allgenes)}
    
    for j, output_file_list in enumerate(alllists):
        for k, cluster in enumerate(output_file_list):
            for gene in cluster:
                res_matrix[gene_indices[gene], j] = k + 1
    
    patterns = ["_".join(map(str, res_matrix[i, :-1])) for i in range(len(allgenes))]
    res_matrix[:, -1] = np.array(patterns, dtype=str)
    
    porcentajes = np.linspace(0, 100, num=21)
    
    for i, percentage in enumerate(np.round(np.linspace(0, 100, len(allgenes)), 4)):
        if percentage in porcentajes:
            print(f"Progress: {percentage}%")
    
    print("Gene/Community matrix calculated, calculating Hamming distances for all gene pairs. This process may take a while...")
    
    gene_community_matrix = res_matrix[:, :-1].astype(int)
    genes_same_communities = {pattern: [] for pattern in np.unique(gene_community_matrix[:,-1])}
    
    for i, pattern in enumerate(gene_community_matrix[:,-1]):
        genes_same_communities[pattern].append(allgenes[i])
    
    with ThreadPoolExecutor(max_workers=threads) as executor:
        distance_matrix = squareform(pdist(gene_community_matrix, metric=distmethod))
    
    final_output = {
        "gene_community_matrix": gene_community_matrix,
        "l_constant": genes_same_communities,
        "distance_matrix": distance_matrix
    }
    
    return final_output


In [ ]:
# add "../../Python/" to path
sys.path.append("../../Python/")

prefix = "../../data/networks-tiny/"
input_layers = [prefix + "interactom.csv",
                prefix + "reactom.csv",
                prefix + "recon_3d.csv"]

CmmD(nodelist = None,
     input_layers = input_layers,
     resolution_start = 0,
     resolution_end = 2,
     interval = 1,
     destfile_community_analysis = "Output/",
     distmethod = "hamming",
     threads = 6)
